# LRU Cache
Design and implement a data structure for the Least Recently Used (LRU) cache that supports the following operations.
- **LRUCache**(capacity: int) -> Initialize an LRU cache with the specified capacity.
- **get**(key: int) -> int: Return the value associated with a key. Return -1 if the key doesn't exist.
- **put**(key: int, value: int) -> None: Add a key and its value to the cache. If adding the key would result in the cache exceeding its size capacity, evict the least recently used element. If the key already exists in the cache, update its value. 

## Intuition

Let's clearly understand the problem. Given a cache that holds **3 elements**, when it reaches **full capacity**, inserting a new value requires two operations:

1. **Eviction**: Remove a key-value pair from the **least recently used (LRU)** end of the cache.
2. **Insertion**: Add a key-value pair to the **most recently used (MRU)** end of the cache.

Retrieving a value also involves two operations:

3. **Update Usage**: Move a key-value pair to the **most recently used** end of the cache.
4. **Lookup**: Access a value using its key.

Now that we've narrowed the design down to these **core operations**, we can determine which **data structures** best support them.

---

### Choosing the Right Data Structures

#### **Operations 1 & 2: Efficient Insertions & Removals**
We need to:
- **Remove** a key-value pair from **one end** of a data structure.
- **Add** a key-value pair to **the other end**.

A **linked list** is well-suited for this, as we can add and remove nodes in **constant time (O(1))** if we have a reference to the node.

**Singly vs. Doubly Linked List:**
- **Adding/removing a node from the head**:  
  - **O(1)** time for both **singly** and **doubly** linked lists.
- **Removing the tail node**:  
  - **O(n)** for a **singly** linked list (even with a tail reference, we must traverse to the previous node).  
  - **O(1)** for a **doubly** linked list (each node stores a reference to its previous node).

Thus, we choose a **doubly linked list**.

To efficiently reference both ends of the list, we define:
- **Head** → Least Recently Used (**LRU**) node.
- **Tail** → Most Recently Used (**MRU**) node.

---

#### **Operations 3 & 4: Fast Key Lookup & Reordering**
- We need to **move** a node to the **MRU end**, regardless of its current position.  
  - If the node is **in the middle**, we’d have to traverse the list to find it.
  - Since each node is associated with a **key**, we can use a **hash map** (dictionary) to store **key → node** mappings.
  - This allows us to access a node in **O(1)** time.

- A **hash map** also efficiently handles **Operation 4**, allowing **constant-time key lookups**.

---

### **Implementation of `put()` and `get()`**

#### **`put(key: int, val: int) -> None`**
1. **Is the key already in the cache?**  
   - **Yes**:  
     - Retrieve the node from the **hash map**.  
     - Remove it from the **linked list** (since we'll update its position).  
   - **No**:  
     - If adding a new node **won't exceed capacity**, add it as the **MRU node** and store it in the **hash map**.  
     - **If the cache is full**:  
       - Remove the **LRU node** from the linked list.  
       - Delete it from the **hash map**.  
       - Add the new node as the **MRU node**.

#### **`get(key: int) -> int`**
1. **Is the key in the cache?**  
   - **No** → Return `-1`.  
   - **Yes**:
     - Retrieve the node from the **hash map**.
     - Move it to the **MRU end**:
       - Remove it from its current position.
       - Add it as the **most recently used** node.
     - Return the **associated value**.

In [ ]:
class DoublyLinkedListNode:
    def __init__(self, key: int, val: int):
        self.key = key
        self.val = val
        self.next = self.prev = None

class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.hashmap = {}
        self.head = DoublyLinkedListNode(-1, -1)
        self.tail = DoublyLinkedListNode(-1, -1)
        self.head.next = self.tail
        self.tail.prev = self.head

    def remove_node(self, node: DoublyLinkedListNode) -> None:
        node.prev.next = node.next
        node.next.prev = node.prev
    
    def add_to_tail(self, node: DoublyLinkedListNode) -> None:
        prev_node = self.tail.prev
        node.prev = prev_node
        node.next = self.tail
        prev_node.next = node
        self.tail.prev = node

    def get(self, key: int) -> int:
        if key not in self.hashmap:
            return -1
        
        self.remove_node(self.hashmap[key])
        self.add_to_tail(self.hashmap[key])
        return self.hashmap[key].val

    def put(self, key: int, value: int) -> None:
        if key in self.hashmap:
            self.remove_node(self.hashmap[key])
        
        node = DoublyLinkedListNode(key, value)
        self.hashmap[key] = node

        if len(self.hashmap) > self.capacity:
            del self.hashmap[self.head.next.key]
            self.remove_node(self.head.next)
        self.add_to_tail(node)

The time complexity for the helper functions remove_node and add_tail_node is O(1) because they perform constant-time operations on a doubly linked list. The put and get functions utilize these helper functions, while also performing constant-time hash map operations. Consequently, they also have an O(1) time complexity.

The overall space complexity is O(n), where n is the capacity of the cache. Both the doubly linked list and hash map can each occupy O(n) space.